# Convolutional Neural Networks

## Load Data

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import KFold

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


2024-07-28 21:04:16.074077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 21:04:16.087331: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 21:04:16.087350: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 21:04:16.096854: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-28 21:04:16.758427: W tensorflow/compiler/tf

Num GPUs Available:  1


2024-07-28 21:04:17.633457: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 21:04:17.667029: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 21:04:17.669876: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
images_train = np.expand_dims(np.load("../data/images/images_train.npy") / 255.0, -1)
images_test = np.expand_dims(np.load("../data/images/images_test.npy") / 255.0, -1)
num_samples_train = images_train.shape[0]
num_samples_test = images_test.shape[0]

y_train = np.load("../data/images/label_ids_train.npy")
y_test = np.load("../data/images/label_ids_test.npy")

train_indices = np.random.permutation(num_samples_train)
images_train = images_train[train_indices]
y_train = y_train[train_indices]

test_indices = np.random.permutation(num_samples_test)
images_test = images_test[test_indices]
y_test = y_test[test_indices]



In [ ]:
labels_train = np.load("../data/images/labels_train.npy")
class_lookup = list(set(zip(y_train, labels_train)))
class_lookup = {id_value: class_value for id_value, class_value in class_lookup}
class_lookup


{4: 'dislocation',
 2: 'greenstick',
 8: 'impacted',
 9: 'avulsion',
 1: 'spiral',
 6: 'longitudinal',
 7: 'oblique',
 3: 'comminuted',
 5: 'pathological',
 0: 'hairline'}

In [ ]:
image_input = layers.Input(shape=(images_train.shape[1], images_train.shape[2], images_train.shape[3]))
x = layers.Conv2D(60, (3, 3), activation='relu')(image_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(120, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)

x = layers.Dense(40, activation='relu')(x)
output = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=[image_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit data to model
history = model.fit(images_train[train], y_train[train],
            batch_size=32,
            epochs=5,
            verbose=1)


In [ ]:
# # Define the K-fold Cross Validator
# kfold = KFold(n_splits=3, shuffle=True)

# # K-fold Cross Validation model evaluation
# fold_no = 1
# acc_per_fold = []
# loss_per_fold = []
# for train, test in kfold.split(images_train, y_train):

#     image_input = layers.Input(shape=(images_train.shape[1], images_train.shape[2], images_train.shape[3]))
#     x = layers.Conv2D(60, (3, 3), activation='relu')(image_input)
#     x = layers.MaxPooling2D((2, 2))(x)
#     x = layers.Conv2D(120, (3, 3), activation='relu')(x)
#     x = layers.MaxPooling2D((2, 2))(x)
#     x = layers.Flatten()(x)
    
#     x = layers.Dense(40, activation='relu')(x)
#     output = layers.Dense(10, activation='softmax')(x)

#     model = models.Model(inputs=[image_input], outputs=output)
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#     # Generate a print
#     print('------------------------------------------------------------------------')
#     print(f'Training for fold {fold_no} ...')
    
#     # Fit data to model
#     history = model.fit(images_train[train], y_train[train],
#               batch_size=32,
#               epochs=5,
#               verbose=1)
    
#     # Generate generalization metrics
#     scores = model.evaluate(images_train[test], y_train[test], verbose=0)
#     print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
#     acc_per_fold.append(scores[1] * 100)
#     loss_per_fold.append(scores[0])
    
#     # Increase fold number
#     fold_no = fold_no + 1


2024-07-28 21:04:18.071907: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 21:04:18.074601: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 21:04:18.076795: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5


I0000 00:00:1722215060.687718   49297 service.cc:145] XLA service 0x7023540038c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722215060.687773   49297 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-28 21:04:20.708595: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-28 21:04:20.800954: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902
2024-07-28 21:04:29.151330: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng46{k2=0,k5=2,k14=4} for conv (f32[60,1,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,1,512,512]{3,2,1,0}, f32[32,60,510,510]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation

 2/21 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - accuracy: 0.0625 - loss: 8.5148

I0000 00:00:1722215077.151889   49297 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20/21 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.1160 - loss: 8.4314

2024-07-28 21:04:44.909545: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng46{k2=0,k5=2,k14=4} for conv (f32[60,1,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[19,1,512,512]{3,2,1,0}, f32[19,60,510,510]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-07-28 21:04:45.148112: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.238631916s
Trying algorithm eng46{k2=0,k5=2,k14=4} for conv (f32[60,1,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[19,1,512,512]{3,2,1,0}, f32[19,60,510,510]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn

21/21 ━━━━━━━━━━━━━━━━━━━━ 30s 638ms/step - accuracy: 0.1171 - loss: 8.1117
Epoch 2/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.2424 - loss: 2.1674
Epoch 3/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.4375 - loss: 1.7388
Epoch 4/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.7209 - loss: 1.0109
Epoch 5/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.9194 - loss: 0.3817
Score for fold 1: loss of 3.7139666080474854; compile_metrics of 26.66666805744171%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 135ms/step - accuracy: 0.1021 - loss: 8.0677
Epoch 2/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.1420 - loss: 2.2815
Epoch 3/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.2304 - loss: 2.1209
Epoch 4/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.4439 - loss: 1.6455
Epoch 5/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.7252 -

2024-07-28 21:05:27.228149: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng46{k2=0,k5=2,k14=4} for conv (f32[60,1,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[20,1,512,512]{3,2,1,0}, f32[20,60,510,510]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-07-28 21:05:27.459908: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.231833438s
Trying algorithm eng46{k2=0,k5=2,k14=4} for conv (f32[60,1,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[20,1,512,512]{3,2,1,0}, f32[20,60,510,510]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn

21/21 ━━━━━━━━━━━━━━━━━━━━ 15s 678ms/step - accuracy: 0.0962 - loss: 10.9666
Epoch 2/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.1811 - loss: 2.2685
Epoch 3/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.3096 - loss: 2.1270
Epoch 4/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.4852 - loss: 1.6075
Epoch 5/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.7768 - loss: 0.8818
Score for fold 3: loss of 2.609377861022949; compile_metrics of 24.012157320976257%


In [ ]:
y_pred = model.predict([images_test])
y_pred = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
report.index = report.index.map(lambda x: class_lookup[int(x)] if x.isdigit() else x)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
report


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 285ms/step
Accuracy: 0.17857142857142858
Classification Report:


,precision,recall,f1-score,support
hairline,0.000000,0.000000,0.000000,10.000000
spiral,0.200000,0.166667,0.181818,12.000000
greenstick,0.277778,0.312500,0.294118,16.000000
comminuted,0.166667,0.285714,0.210526,14.000000
dislocation,0.272727,0.315789,0.292683,19.000000
pathological,0.176471,0.166667,0.171429,18.000000
longitudinal,0.000000,0.000000,0.000000,12.000000
oblique,0.100000,0.062500,0.076923,16.000000
impacted,0.166667,0.111111,0.133333,9.000000
avulsion,0.214286,0.214286,0.214286,14.000000
